# Overview: Manual testing for plotting functionality

Notes:
- Dimensions are not `x` and `y` since this is used internally in matplotlib-related code and masks potential bugs.

In [ ]:
import scipp as sc
import numpy as np
def make_array(unit='K'):
    data = sc.array(dims=['xx','yy','zz'], unit=unit, values=np.sin(0.1*np.arange(4*4*40)).reshape(4,40,4), variances=0.01*np.ones((4,40,4)))
    da = sc.DataArray(data=data)
    da.coords['yy'] = sc.linspace(dim='yy', unit='km', start=1.1, stop=5.2, num=40)
    da.coords['xx'] = sc.linspace(dim='xx', unit='mm', start=1.1, stop=5.2, num=5)
    da.coords['xx2'] = sc.linspace(dim='xx', unit='m', start=1.1, stop=55.2, num=5)
    da.masks['mask'] = da.coords['xx']['xx',1:] > 5.0 * sc.Unit('mm')
    return da

# 1d plotting
## General

Check:
- Axis switching
  - Log buttons remember state
- Slider (before and after switching)
  - Slider range readout
- Profile (before and after switching)
  - Line labels in profile figure
- Keeping lines (check line labels)

In [ ]:
da = make_array()
da['zz',0].plot(projection='1d', vmin=-2*da.unit, vmax=2*da.unit)

Check:
- Similar to above, but with two sliders.
- Profile button should currently be disabled since 2d profile not supported yet.

In [ ]:
da.plot(projection='1d', vmin=-2*da.unit, vmax=2*da.unit)

## 1d plot of non-counts

Check that normalization not messed up, plot should show a constant line at $1~\text{K}$:

In [ ]:
import scipp as sc
da = sc.DataArray(data=sc.ones(dims=['x'], shape=[100], unit='K'),
                  coords={'x':sc.geomspace(dim='x', start=1, stop=1e6, num=100)})
da.plot()

## 1d plot with multiple lines

Check:
- Lines can be kept and labels are correct
- Profile works
  - Clicking keeps both lines in profile, with correct labels

In [ ]:
da = make_array()
sc.plot({'a':da['zz',0], 'b':da['zz',1]}, projection='1d')

# 2d plotting
## 2d plot of 3d data with slider

- Custom labels for `xx` should be used for:
  - axis tics
  - button for dim selection
  - slider range display
  - line label if line kept in profile plot

In [ ]:
da = make_array()
da.coords['xx2']

In [ ]:
da.plot(labels={'xx':'xx2'}, errorbars=False, resolution=3)

In [ ]:
da.plot(labels={'xx':'xx2'}, errorbars=False, resolution=3, resampling_mode='sum')

## 2d plot of 3d counts data with slider

- Profile readout must be consistent through zoom levels
  - Note that this is only the case as long as pixels are not merged

In [ ]:
da = make_array(unit='counts')
da.plot(labels={'xx':'xx2'}, errorbars=False)

## 2d plot of 4d data

Check:
- Profile button not shown since 2d profile not supported yet
- Axis switching etc. works

In [ ]:
da4d = sc.concatenate(da, da*sc.scalar(2.0), 'time')
da4d.plot()

# Datetime coords

Check (in all of the below plots):
- Axis tick labels correct
- Labels on sliders correct
- Behavior when zooming correct
  - Note crash with certain range (can be triggered by zooming or changing datetime coord to commented line) https://github.com/scipp/scipp/issues/2021

In [ ]:
da = make_array()
da.coords['xx'] = sc.scalar(np.datetime64("now")) + sc.linspace(dim='xx', unit='s', dtype='int64', start=1, stop=5000, num=5)
# TODO fix datetime formatting exception https://github.com/scipp/scipp/issues/2021
# Trigger by zoom to certain level or use this:
#da.coords['xx'] = sc.scalar(np.datetime64("now")) + sc.linspace(dim='xx', unit='s', dtype='int64', start=1, stop=3, num=5)
da['zz',0].plot(projection='1d')

In [ ]:
da['zz',0]['yy',0].plot()

In [ ]:
da.plot()

In [ ]:
da['zz', 0].plot()

Check correct bevahior with a higher-precision datetime, especially when zooming in:

In [ ]:
da.coords['xx'] = sc.to_unit(da.coords['xx'], 'ns')
da.plot()

# `redraw`

In [ ]:
da1d = da['xx',0]['zz',0].copy()
plot = da1d.plot()
plot.show()

Running this cell should flip data (vertical axis) in plot above:

In [ ]:
da1d.data *= -1.0
plot.redraw()

In [ ]:
da2d = da['xx',0].copy()
da2d.masks['mask'] = da2d.coords['yy'] < 2.0 * sc.Unit('km')
plot = da2d.plot()
plot.show()

Running this cell should flip data (color axis) in plot above, and the masked area should move as well:

In [ ]:
da2d.data *= -1.0
da2d.masks['mask'] ^= da2d.coords['yy'] < 3.0 * sc.Unit('km')
plot.redraw()

# Multi-dimensional coord

In [ ]:
da_multi = da.copy()
da_multi.coords['zz'] = sc.array(dims=['yy', 'zz'], unit='um', values=np.arange(160).reshape(40,4))
da_multi['xx',0].plot()

In [ ]:
da_multi.plot()

In [ ]:
try:
    da_multi.transpose().plot()
except sc.DimensionError as e:
    print(e)

# Binned data

In [ ]:
import scipp as sc
import numpy as np
N = int(80000)
values = np.random.rand(N)
data = sc.DataArray(
    data=sc.Variable(dims=['event'], values=values, unit=''),
    coords={
        'xx':sc.Variable(dims=['event'], values=np.random.rand(N)),
        'yy':sc.Variable(dims=['event'], values=np.random.rand(N)),
        'zz':sc.Variable(dims=['event'], values=np.random.rand(N))
    })
binned = sc.bin(data, edges=[sc.linspace(dim='xx', start=0.0,stop=1.0,num=10),
                             sc.linspace(dim='yy', start=0.0,stop=1.0,num=4),
                             sc.linspace(dim='zz', start=0.0,stop=1.0,num=4)])
binned.plot(resolution=100)

In [ ]:
binned['xx', 0].plot(resolution=100)

Known issues (to fix):
- Slider steps are based on coarse underlying binning
- Profile selection based on coarse underlying binning

In [ ]:
binned['xx', 0].plot(projection='1d', resolution=100)

In [ ]:
binned['xx', 0]['yy', 0].plot()

# 3d plotting
Known issues (to fix):
- Profile button should be disabled or not shown, until supported

## Existing positions

In [ ]:
import scipp as sc
import numpy as np
N = 1000
M = 100
theta = np.random.random(N) * np.pi
phi = np.random.random(N) * 2.0 * np.pi
r = 10.0 + (np.random.random(N) - 0.5)
x = r * np.sin(theta) * np.sin(phi)
y = r * np.sin(theta) * np.cos(phi)
z = r * np.cos(theta)

a = np.arange(2*M*N).reshape([2, M, N]) * np.sin(y)
pos = sc.vectors(dims=['xyz'], unit='m', values=np.array([x, y, z]).T)
pos.fields.y *= 2
da = sc.DataArray(
    data=sc.array(dims=['abc', 'time', 'xyz'], values=a, unit='K'),
    #masks={
    #    'mask':pos.fields.x < 0.0 * sc.Unit('m')},
    coords={
        'xyz':pos,
        'time':sc.array(dims=['time'], values=np.arange(M).astype(np.float))})
da.plot(projection="3d", positions='xyz')

## Fake positions for dense data

In [ ]:
import scipp as sc
import numpy as np
N = 10
M = 40
L = 30
K = 20
xx = np.arange(3, N, dtype=np.float64)
yy = np.arange(4, M, dtype=np.float64)
zz = np.arange(5, L, dtype=np.float64)
qq = np.arange(6, K, dtype=np.float64)
x, y, z, q = np.meshgrid(xx, yy, zz, qq, indexing='ij')
b = N/20.0
c = M/2.0
d = L/2.0
r = np.sqrt(((x-c)/b)**2 + ((y-c)/b)**2 + ((z-d)/b)**2  + ((q-d)/b)**2)
a = np.sin(r)
d = sc.Dataset()
da = sc.DataArray(data=sc.Variable(dims=['xx', 'yy', 'zz', 'Q_x'], values=a, unit='K'))
da.coords['xx'] = sc.Variable(dims=['xx'], values=xx)
da.coords['yy'] = sc.Variable(dims=['yy'], values=yy, unit='m')
da.coords['zz'] = sc.Variable(dims=['zz'], values=zz)
da.coords['Q_x'] = sc.Variable(dims=['Q_x'], values=qq)

xx2 = da.coords['xx'].copy()
yy2 = da.coords['yy'].copy()
zz2 = da.coords['zz'].copy()
yy2.unit = ''  # deal with potential different units
xx2 *= 3.0  # deal with potential differences in axis scales
da.coords['dummy-pos'] = sc.geometry.position(xx2, yy2, zz2)
da.plot(projection='3d', positions='dummy-pos')